# Data Preparation

## Setup

In [1]:
# Data handling
import pandas as pd
import polars as pl
import glob
import os

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

## 1. Load Data
First, we will use the function 'glob' because it will return all the files starting with "merged".

In [4]:
# Merged Files Path
path = "../Data/Raw"

# Grabbing the correct path
raw_data_path = os.path.join(path, "merged*.csv")

# Putting all the chunks in an array
merged_files = sorted(glob.glob(raw_data_path))

print("Found Files: ", len(merged_files))

Found Files:  63


Using Polars to handle big datasets

In [5]:
# Using LazyFrames for each CSV
lazy_frames = [pl.scan_csv(f) for f in merged_files]

# Concatenating all files into one dataset
raw_concat_df = pl.concat(lazy_frames)

# Running this method to create one single dataset
raw_concat = raw_concat_df.collect()

# Checking result
print(raw_concat.head(2))

shape: (2, 40)
┌────────────┬────────────┬────────────┬────────────┬───┬──────────┬────────┬──────────┬───────────┐
│ Header_Len ┆ Protocol   ┆ Time_To_Li ┆ Rate       ┆ … ┆ IAT      ┆ Number ┆ Variance ┆ Label     │
│ gth        ┆ Type       ┆ ve         ┆ ---        ┆   ┆ ---      ┆ ---    ┆ ---      ┆ ---       │
│ ---        ┆ ---        ┆ ---        ┆ f64        ┆   ┆ f64      ┆ i64    ┆ f64      ┆ str       │
│ f64        ┆ i64        ┆ f64        ┆            ┆   ┆          ┆        ┆          ┆           │
╞════════════╪════════════╪════════════╪════════════╪═══╪══════════╪════════╪══════════╪═══════════╡
│ 19.92      ┆ 6          ┆ 63.36      ┆ 25893.9622 ┆ … ┆ 0.000039 ┆ 100    ┆ 1772.41  ┆ DDOS-PSHA │
│            ┆            ┆            ┆ 18         ┆   ┆          ┆        ┆          ┆ CK_FLOOD  │
│ 0.0        ┆ 47         ┆ 64.0       ┆ 3703.84133 ┆ … ┆ 0.000271 ┆ 100    ┆ 2304.0   ┆ MIRAI-GRE │
│            ┆            ┆            ┆ 1          ┆   ┆          ┆        

### 1.1 Saving Raw dataset
The format will be Parquet because it is faster than working with CSV files.

In [6]:
raw_concat.write_parquet("../Data/Raw/Raw_Dataset")

### 1.2 Loading Raw Dataset

In [2]:
raw_df = pl.read_parquet("../Data/Raw/Raw_Dataset")

### 1.3 Initial Data Exploration Before Cleaning

In [3]:
raw_df.describe()

statistic,Header_Length,Protocol Type,Time_To_Live,Rate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,ack_flag_number,ece_flag_number,cwr_flag_number,ack_count,syn_count,fin_count,rst_count,HTTP,HTTPS,DNS,Telnet,SMTP,SSH,IRC,TCP,UDP,DHCP,ARP,ICMP,IGMP,IPv,LLC,Tot sum,Min,Max,AVG,Std,Tot size,IAT,Number,Variance,Label
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""count""",4.5019243e7,4.5019243e7,4.5019243e7,4.5019243e7,4.5019242e7,4.5019242e7,4.5019241e7,4.5019241e7,4.5019241e7,4.5019241e7,4.5019241e7,4.5019241e7,4.5019241e7,4.5019241e7,4.5019241e7,4.5019241e7,4.5019241e7,4.501924e7,4.501924e7,4.501924e7,4.501924e7,4.501924e7,4.5019239e7,4.5019239e7,4.5019239e7,4.5019237e7,4.5019237e7,4.5019237e7,4.5019236e7,4.5019236e7,4.5019236e7,4.5019236e7,4.5019236e7,4.5019236e7,4.5018566e7,4.5019236e7,4.5019236e7,4.5019235e7,4.5018564e7,"""45019234"""
"""null_count""",0.0,0.0,0.0,0.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,6.0,6.0,6.0,7.0,7.0,7.0,7.0,7.0,7.0,677.0,7.0,7.0,8.0,679.0,"""9"""
"""mean""",13.737131,9.091308,66.531692,inf,0.08692,0.206446,0.092911,0.093867,0.129789,0.000028,0.000015,9.856238,20.399865,8.608305,9.171135,0.050231,0.058763,0.002632,0.000011,0.000011,0.000206,0.000016,0.574688,0.21731,0.000175,0.00283,0.163487,0.000021,0.99717,0.99717,10953.114966,79.958927,223.347508,131.428351,41.311501,131.428351,0.010303,95.51115,34246.200863,null
"""std""",8.72476,9.09442,14.416833,NaN,0.27934,0.399469,0.2833,0.276981,0.316596,0.002022,0.001403,28.020519,39.90779,27.926966,28.226764,0.213406,0.219492,0.022176,0.000693,0.000768,0.007574,0.000999,0.484838,0.401024,0.004795,0.018719,0.366346,0.001318,0.018719,0.018719,16852.252262,107.172392,582.585731,229.087023,180.38725,229.087023,21.176433,19.555591,388353.613362,null
"""min""",0.0,0.0,0.0,0.000013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,42.0,46.0,46.0,0.0,46.0,-0.017818,1.0,0.0,"""BACKDOOR_MALWARE"""
"""25%""",8.0,6.0,64.0,11379.321197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,6000.0,60.0,60.0,60.0,0.0,60.0,0.000027,100.0,0.0,null
"""50%""",20.0,6.0,64.0,24600.02346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.99,0.0,0.0,0.0,0.0,0.0,1.0,1.0,6000.0,60.0,60.0,60.0,0.0,60.0,0.000041,100.0,0.0,null
"""75%""",20.0,17.0,64.0,37936.903039,0.0,0.01,0.0,0.0,0.01,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.03,0.0,0.0,0.0,0.0,1.0,1.0,6010.0,60.0,74.0,60.25,0.844232,60.25,0.000089,100.0,0.712727,null
"""max""",60.0,47.0,255.0,inf,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0,100.0,100.0,100.0,1.0,1.0,1.0,0.6,0.9,1.0,0.9,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,316492.0,13583.0,52194.0,13583.0,11655.404669,13583.0,78612.003899,100.0,1.35848458e8,"""XSS"""


This dataset contains around **45 million rows and 40 columns**. The target variable is **"label"** and the rest of the columns are type f64(float). Less than 10 missing values appear in most columns which is very small. However, exists some infinite (inf) values that will need to be treated.

## 3. Data Cleaning

Checking which columns contain null values

In [4]:
raw_df.select([
    pl.col(c).is_null().sum().alias(c) for c in raw_df.columns
])

Header_Length,Protocol Type,Time_To_Live,Rate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,ack_flag_number,ece_flag_number,cwr_flag_number,ack_count,syn_count,fin_count,rst_count,HTTP,HTTPS,DNS,Telnet,SMTP,SSH,IRC,TCP,UDP,DHCP,ARP,ICMP,IGMP,IPv,LLC,Tot sum,Min,Max,AVG,Std,Tot size,IAT,Number,Variance,Label
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,1,1,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,4,4,4,6,6,6,7,7,7,7,7,7,677,7,7,8,679,9


Dropping null values in 'Label' and 'Std' columns 

In [5]:
raw_df = raw_df.drop_nulls(["Label", "Std"])

Removing Duplicates

In [6]:
raw_df = raw_df.unique()

Dropping Infinite Values

In [7]:
raw_df = raw_df.filter(~pl.col("Rate").is_infinite())

In [8]:
raw_df.describe()

statistic,Header_Length,Protocol Type,Time_To_Live,Rate,fin_flag_number,syn_flag_number,rst_flag_number,psh_flag_number,ack_flag_number,ece_flag_number,cwr_flag_number,ack_count,syn_count,fin_count,rst_count,HTTP,HTTPS,DNS,Telnet,SMTP,SSH,IRC,TCP,UDP,DHCP,ARP,ICMP,IGMP,IPv,LLC,Tot sum,Min,Max,AVG,Std,Tot size,IAT,Number,Variance,Label
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""count""",2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,2.100524e7,"""21005240"""
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0"""
"""mean""",14.022944,11.235932,69.413359,19238.575991,0.061738,0.19504,0.074058,0.093043,0.168224,0.000061,0.000031,10.279988,18.98586,5.994794,7.158183,0.065235,0.096103,0.00564,0.000022,0.000022,0.000441,0.000034,0.550377,0.245321,0.000375,0.005679,0.113021,0.000045,0.994321,0.994321,16065.206757,97.256727,404.153965,207.398039,88.3685,207.398039,0.022037,90.555317,73295.144494,null
"""std""",9.085971,11.468747,20.664064,31067.467221,0.234749,0.384731,0.245408,0.258343,0.336026,0.00296,0.002054,26.545545,38.374556,23.437795,24.282325,0.237366,0.267552,0.032201,0.001012,0.001122,0.011083,0.00146,0.477123,0.406701,0.007011,0.027051,0.3081,0.001917,0.027051,0.027051,23160.752796,147.745118,814.40915,315.032966,255.90263,315.032966,31.001877,27.526711,565975.365401,null
"""min""",0.0,0.0,0.0,0.000013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0,42.0,46.0,46.0,0.0,46.0,-0.017818,2.0,0.0,"""BACKDOOR_MALWARE"""
"""25%""",7.92,6.0,64.0,4503.08021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,6000.0,60.0,60.0,60.0,0.0,60.0,0.000038,100.0,0.0,null
"""50%""",19.8,6.0,64.0,12682.341558,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.0,0.0,0.0,0.0,0.0,1.0,1.0,6014.0,60.0,78.0,60.3,1.4,60.3,0.00008,100.0,1.96,null
"""75%""",20.0,17.0,65.73,27299.557407,0.0,0.01,0.0,0.01,0.03,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,1.0,0.4,0.0,0.0,0.0,0.0,1.0,1.0,6665.0,60.0,441.0,134.1,29.408135,134.1,0.000225,100.0,864.838384,null
"""max""",60.0,47.0,255.0,1.572864e7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0,100.0,100.0,100.0,1.0,1.0,1.0,0.6,0.9,1.0,0.9,1.0,1.0,0.8,1.0,1.0,0.4,1.0,1.0,316492.0,7306.0,52194.0,9430.3,11655.404669,9430.3,78612.003899,100.0,1.35848458e8,"""XSS"""


Now the dataset contains around **21 million rows and 40 columns**. Also, there are no missing values either duplicates.

## 4. Feature Selection

Checking highly correlated feature pairs

In [9]:
sample_df = raw_df.sample(n=500_000, shuffle=True).to_pandas()
corr = sample_df.corr(numeric_only=True).abs()

# Columns with strong correlations (|r| > 0.95)
high_corr = [(i, j) for i in corr.columns for j in corr.columns
              if i != j and corr.loc[i, j] > 0.95]
high_corr

[('fin_flag_number', 'fin_count'),
 ('syn_flag_number', 'syn_count'),
 ('rst_flag_number', 'rst_count'),
 ('syn_count', 'syn_flag_number'),
 ('fin_count', 'fin_flag_number'),
 ('fin_count', 'rst_count'),
 ('rst_count', 'rst_flag_number'),
 ('rst_count', 'fin_count'),
 ('ARP', 'IPv'),
 ('ARP', 'LLC'),
 ('IPv', 'ARP'),
 ('IPv', 'LLC'),
 ('LLC', 'ARP'),
 ('LLC', 'IPv'),
 ('AVG', 'Tot size'),
 ('Tot size', 'AVG')]

Feature-importance ranking

In [10]:
# Convert to pandas for modeling
df = raw_df.sample(n=1_000_000, shuffle=True).to_pandas()

X = df.drop("Label", axis=1)
y = LabelEncoder().fit_transform(df["Label"])

# Train-test split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a LightGBM model
model = lgb.LGBMClassifier(n_estimators=100, random_state=42,verbose=-1)
model.fit(X_train, y_train)

# Get feature importances
importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
importances.head(20)

Time_To_Live       6317
Header_Length      5847
Rate               5677
IAT                5473
Tot sum            4648
Max                3950
AVG                3188
Std                3065
HTTPS              2337
HTTP               2317
ack_flag_number    2140
syn_flag_number    2013
psh_flag_number    1967
Min                1912
UDP                1898
Variance           1550
ICMP               1436
rst_flag_number    1392
ack_count          1325
TCP                1161
dtype: int32

In [11]:
# Defining redundant pairs
redundant_pairs = [
    ("fin_flag_number", "fin_count"),
    ("syn_flag_number", "syn_count"),
    ("rst_flag_number", "rst_count"),
    ("ARP", "IPv"),
    ("ARP", "LLC"),
    ("IPv", "LLC"),
    ("AVG", "Tot size")
]

# Getting model importance ranking as a pandas Series
importances = model.feature_importances_
features = X.columns
importance_df = pd.DataFrame({"feature": features, "importance": importances})

# Dropping redundant features
to_drop = []
for a, b in redundant_pairs:
    if a in features and b in features:
        # dropping the one with lower importance
        drop_col = a if importance_df.loc[importance_df.feature == a, "importance"].item() < \
                         importance_df.loc[importance_df.feature == b, "importance"].item() else b
        to_drop.append(drop_col)

clean_features = [f for f in features if f not in to_drop]
print("Dropped:", to_drop)
print("Final features:", clean_features)

Dropped: ['fin_count', 'syn_count', 'rst_count', 'IPv', 'LLC', 'LLC', 'Tot size']
Final features: ['Header_Length', 'Protocol Type', 'Time_To_Live', 'Rate', 'fin_flag_number', 'syn_flag_number', 'rst_flag_number', 'psh_flag_number', 'ack_flag_number', 'ece_flag_number', 'cwr_flag_number', 'ack_count', 'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IGMP', 'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'IAT', 'Number', 'Variance']


In [12]:
cols_to_drop = ['fin_count', 'syn_count', 'rst_count', 'IPv', 'LLC', 'Tot size']

raw_df = raw_df.drop(cols_to_drop)

### Saving Cleaned Dataset

In [13]:
raw_df.write_parquet("../Data/Cleaned/Cleaned_Dataset")